In [1]:
import math                       
import matplotlib.pyplot as plt   
import random                     
import numpy as np                
import numpy.linalg as lin        
import scipy.special as ss
import pickle
from cgi import test
from copy import deepcopy

In [2]:
def get_mag(A):
  return np.sum(A)

def distance(A,B):
  distance = sum(np.logical_and(A,B))
  return distance

def perm(A,N):
  return np.roll(A,N)

In [3]:
def u_gen_rand_hv(D,d):
    # Sanity checker
    if (D % 2):
        print("Error - D can't be an odd number")
        return 0
    # generate
    chosen = random.sample(range(D), k =int(D*d))
    rand_hv = np.array([1 if x in chosen else 0 for x in range(D)])
    return rand_hv

def create_item_mem(N,D,d):
  # insert nice code here
  item_mem = {}
  first = u_gen_rand_hv(D,d)
  item_mem.update({0:first})
  last = u_gen_rand_hv(D,d)
  item_mem.update({N-1:last})
  for i in range(1,N-1):
    temp_hv = np.concatenate((first[:int((D/N)*(N-i))],last[int((D/N)*(N-i)):])) 
    item_mem.update({i:temp_hv})    
  return item_mem

def bundle(block,t): # t - threshold
  if((len(block) % 2) == 0):
    print("Error! Block size should be odd.")
    return
  sum = np.sum(block,axis=0)
  bndl = np.where(sum>t,1,0)
  return bndl

In [85]:
Q = 2
D = 1024
d = 0.01

im = create_item_mem(Q,D,d)

print(len(im))

[1, 4, 4, 3, 2]


In [86]:
h = u_gen_rand_hv(20,0.5)
print(h)
chs = random.sample(range(20), k =int(5))
hi = [not h[x] if x in chs else h[x] for x in range(20)]
print(hi)
print(np.array(hi))

[1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 0]
[1, 0, False, True, True, 1, True, 1, 1, 1, 0, 0, 1, False, 0, 0, 1, 0, 1, 0]
[1 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0]


In [87]:
def hdc_encode(voice,voice_im,D,d,Q=10, remove_list = []): #d = density, Q - quntization steps
    voice = [ int(math.floor(x*(Q/2))+(Q/2)) if x<1 else int(math.floor((x-0.0001)*(Q/2))+(Q/2)) for x in voice]
    feature_hv_list = [voice_im[x] for x in voice]
    feature_hv_list = np.array([perm(feature_hv_list[x],x) for x in range(len(feature_hv_list))])
    
    threshold = 36 #first threshold
    bundle = np.sum(feature_hv_list,axis=0)
    
    if remove_list:
        np.delete(bundle, remove_list)
        
    #for threshold in range(1000):
    bndl = np.where(bundle>threshold,1,0)
    #    print(threshold,sum(bndl))
    return bndl

In [88]:
def dimension_sparsifier(non_bin_reg, dimensionality, sparsity, remove_list):
  remove_num = int(sparsity * dimensionality)
  print(remove_num)
  max_list = non_bin_reg[0]
  min_list = non_bin_reg[0]
  
  for i in range(1,len(non_bin_reg)):
    max_list = np.maximum(max_list, non_bin_reg[i])
    min_list = np.where(np.logical_or(min_list>=non_bin_reg[i], min_list==0), non_bin_reg[i], min_list)

  var_list = np.subtract(max_list, min_list)
  var_list = sorted(enumerate(var_list), key=lambda x:x[1])
  idx, value = map(list, zip(*var_list))
  remove_list = idx[0:remove_num]

  for i in range(len(non_bin_reg)):
    np.delete(non_bin_reg[i], remove_list)

In [89]:
D = 5000
Q = 10
d = 0.05
remove_list = []

voice_im = create_item_mem(Q,D,d)
for i in range(10):
    print(i,distance(voice_im[0],voice_im[i]))

with open('isolet.pkl', 'rb') as f:
    isolet = pickle.load(f)
trainData, trainLabels, testData, testLabels = isolet
trainData = np.array(trainData)
testData = np.array(testData)

keys = [x for x in range(26)]
voice_am = dict()

0 250
1 223
2 206
3 185
4 150
5 132
6 105
7 86
8 58
9 10


In [90]:
for i in range(1):
    print(sum(hdc_encode(trainData[65],voice_im,D,d,Q)))

549


In [91]:
non_bin_reg = np.zeros((26,D))
#print(hdc_encode(trainData[i],voice_im, D, d, Q))
for i in range(len(trainLabels)):
    non_bin_reg[trainLabels[i]] = np.add(non_bin_reg[trainLabels[i]],hdc_encode(trainData[i],voice_im, D, d, Q))

    
print(non_bin_reg)

dimension_sparsifier(non_bin_reg, D, d, remove_list)

[[  9.  49. 114. ...   1.   0.   2.]
 [  4.  27.  92. ...  12.   0.   0.]
 [  0.   5.  34. ...   7.   4.   0.]
 ...
 [  0.   0.  16. ...   0.   0.   0.]
 [  6.  11.  32. ...   0.   0.   1.]
 [ 15.  10.  32. ...   5.   7.   4.]]
250


In [92]:
for threshold2 in range(500):
    print(threshold2,sum(np.where(non_bin_reg[0]>threshold2,1,0)))

0 2574
1 2304
2 2167
3 2049
4 1959
5 1885
6 1816
7 1759
8 1702
9 1641
10 1597
11 1551
12 1507
13 1473
14 1431
15 1392
16 1367
17 1335
18 1317
19 1292
20 1272
21 1245
22 1224
23 1212
24 1198
25 1176
26 1158
27 1136
28 1120
29 1090
30 1077
31 1056
32 1040
33 1024
34 1014
35 1002
36 986
37 967
38 958
39 942
40 931
41 917
42 908
43 894
44 878
45 868
46 860
47 843
48 834
49 820
50 813
51 804
52 793
53 779
54 770
55 759
56 756
57 748
58 738
59 732
60 720
61 708
62 701
63 690
64 681
65 671
66 667
67 663
68 661
69 648
70 638
71 628
72 624
73 617
74 608
75 599
76 590
77 582
78 576
79 570
80 563
81 561
82 554
83 550
84 545
85 540
86 532
87 525
88 517
89 512
90 502
91 499
92 494
93 487
94 482
95 473
96 471
97 465
98 460
99 457
100 450
101 446
102 440
103 432
104 427
105 423
106 417
107 412
108 407
109 402
110 394
111 387
112 381
113 378
114 373
115 365
116 362
117 359
118 353
119 345
120 342
121 340
122 333
123 331
124 329
125 323
126 318
127 312
128 312
129 307
130 299
131 297
132 295
133 289
13

In [93]:
threshold2 = 40 #second threshold
for i in range(len(keys)):
    voice_am[keys[i]] = np.where(non_bin_reg[i]>threshold2,1,0)
    print(i,sum(voice_am[keys[i]]))

0 931
1 945
2 927
3 938
4 924
5 986
6 921
7 927
8 931
9 971
10 946
11 938
12 985
13 964
14 909
15 928
16 964
17 871
18 974
19 938
20 931
21 950
22 998
23 986
24 897
25 932


In [94]:
def similarity_search(voice,voice_im,voice_am,D,d,Q):
  # insert nice code here
  sim_score = 0
  sim_letter = '0'
  test_hv = hdc_encode(voice,voice_im,D,d,Q, remove_list)
  for each in voice_am.items():
    #print(each)
    if (distance(test_hv,each[1])>sim_score):
      sim_score,sim_letter = distance(test_hv,each[1]), each[0] 
  return sim_letter, sim_score

In [95]:
def test_model(test_data,correct_values,voice_im,voice_am,D,d,Q,print_flag):

  # Simply iterate through all elements in the clean_letters set
  score = 0
  test_len = len(test_data)
  for i in range(test_len):
    
    sim_letter, sim_score = similarity_search(test_data[i],voice_im,voice_am,D,d,Q)

    
    if sim_letter == correct_values[i]:
      score += 1
      if(print_flag):
        print("CORRECT prediction! sim_letter: ", sim_letter, " sim_score: ", str(sim_score))
    else:
      if(print_flag):
        print("WRONG prediction! sim_letter: " , sim_letter, " sim_score: ", str(sim_score))

  print("Final accuracy is: %f" % (score/test_len)*100))
  return (score/test_len)*100

In [96]:
test_model(testData,testLabels,voice_im,voice_am,D,d,Q,True)

CORRECT prediction! sim_letter:  0  sim_score:  432
CORRECT prediction! sim_letter:  0  sim_score:  402
WRONG prediction! sim_letter:  4  sim_score:  401
CORRECT prediction! sim_letter:  1  sim_score:  414
CORRECT prediction! sim_letter:  2  sim_score:  411
CORRECT prediction! sim_letter:  2  sim_score:  419
WRONG prediction! sim_letter:  19  sim_score:  421
CORRECT prediction! sim_letter:  3  sim_score:  407
CORRECT prediction! sim_letter:  4  sim_score:  407
CORRECT prediction! sim_letter:  4  sim_score:  378
CORRECT prediction! sim_letter:  5  sim_score:  382
CORRECT prediction! sim_letter:  5  sim_score:  394
WRONG prediction! sim_letter:  19  sim_score:  402
WRONG prediction! sim_letter:  19  sim_score:  423
WRONG prediction! sim_letter:  4  sim_score:  366
CORRECT prediction! sim_letter:  7  sim_score:  411
CORRECT prediction! sim_letter:  8  sim_score:  417
CORRECT prediction! sim_letter:  8  sim_score:  408
CORRECT prediction! sim_letter:  9  sim_score:  437
CORRECT prediction!

82.74534958306606